<a href="https://colab.research.google.com/github/TING-JHEN/SimCSE_chinese/blob/main/SNLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

data:  https://github.com/pluto-junzeng/CNSD

In [ ]:
!pip install sentence_transformers

In [ ]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import LoggingHandler, SentenceTransformer, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
import datetime
import os
import gzip
import csv
import json
import random

# Training parameters
model_name = 'bert-base-chinese'
num_epochs = 1
max_seq_length = 32
train_data_size = 10000  #最多440484個

In [ ]:
# Check if dataset exsist. If not, download and extract it

train_dataset_path = '/content/SNLI.train.txt'  # ['origin', "entailment", "contradiction"] 有440484個
dev_dataset_path = '/content/STS-B.valid.data'  # ['sentence1', 'sentence2', 'score'] score是兩個句子的關聯性(0-5)
test_dataset_path = '/content/STS-B.test.data'

# train_samples is a list of InputExample objects where we pass the same sentence twice to texts, i.e. texts=[sent, sent]
unsupervised_train_samples = []
supervised_train_samples_without_hard_negative = []
supervised_train_samples_with_hard_negative = []
data = []
with open(train_dataset_path, 'r', encoding='utf8') as fIn:
    for line in fIn:
        line = json.loads(line)
        data.append(line)

if train_data_size > len(data):
    train_data_size = len(data)
data = random.sample(data,train_data_size)  # 隨機取train_data_size個

for line in data:
    if len(line) == 3:
        unsupervised_train_samples.append(InputExample(texts=[line["origin"], line["origin"]]))
        supervised_train_samples_without_hard_negative.append(InputExample(texts=[line["origin"], line["entailment"]]))
        supervised_train_samples_with_hard_negative.append(InputExample(texts=[line["origin"], line["entailment"], line["contradiction"]]))

dev_samples = []
with open(dev_dataset_path, 'r', encoding='utf8') as fIn:
    for line in fIn:
        line = line.strip().split('\t')
        score = float(line[2]) / 5.0  # Normalize score to range 0 ... 1
        if len(line) == 3:
            dev_samples.append(InputExample(texts=[line[0], line[1]],label=score))

test_samples = []
with open(test_dataset_path, 'r', encoding='utf8') as fIn:
    for line in fIn:
        line = line.strip().split('\t')
        score = float(line[2]) / 5.0  # Normalize score to range 0 ... 1
        if len(line) == 3:
            test_samples.append(InputExample(texts=[line[0], line[1]],label=score))

i = 87
print('len(unsupervised_train_samples)=',train_data_size,unsupervised_train_samples[i])
print('len(supervised_train_samples_without_hard_negative)=',train_data_size,supervised_train_samples_without_hard_negative[i])
print('len(supervised_train_samples_with_hard_negative)=',train_data_size,supervised_train_samples_with_hard_negative[i])
print('len(dev_samples)=',len(dev_samples),dev_samples[i])
print('len(test_samples)=',len(test_samples),test_samples[i])

len(unsupervised_train_samples)= 10000 <InputExample> label: 0, texts: 一个穿白绿相间的男人正在球场上踢足球。; 一个穿白绿相间的男人正在球场上踢足球。
len(supervised_train_samples_without_hard_negative)= 10000 <InputExample> label: 0, texts: 一个穿白绿相间的男人正在球场上踢足球。; 有几个男人在踢足球。
len(supervised_train_samples_with_hard_negative)= 10000 <InputExample> label: 0, texts: 一个穿白绿相间的男人正在球场上踢足球。; 有几个男人在踢足球。; 一群人在打棒球。
len(dev_samples)= 1458 <InputExample> label: 0.2, texts: 一个女人正在剥一些鱼。; 一个女人正在煎锅上倒一种黄色的混合物。
len(test_samples)= 1361 <InputExample> label: 0.6, texts: 熊猫躺在圆木上。; 一只熊猫躺着。


In [ ]:
# Here we define our SentenceTransformer model
word_embedding_model = models.Transformer(model_name, max_seq_length=max_seq_length)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
print(model)

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 32, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


In [ ]:
model.save('/content/output/setting_dropout')

In [ ]:
停! 打開 /content/output/setting_dropout/config.json 調整 dropout(預設是0.1)

SyntaxError: ignored

In [ ]:
#選一個train_sample <- [ unsupervised_train_samples, supervised_train_samples_without_hard_negative, supervised_train_samples_with_hard_negative ]
train_batch_size = 8
train_samples = supervised_train_samples_with_hard_negative
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, batch_size=train_batch_size, name='sts-dev')
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, batch_size=train_batch_size, name='sts-test')

In [ ]:
model = SentenceTransformer('/content/output/setting_dropout')

# We train our model using the MultipleNegativesRankingLoss
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size, drop_last=True)
train_loss = losses.MultipleNegativesRankingLoss(model)  # 用這個就可以實現對比學習

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up
evaluation_steps = int(len(train_dataloader) * 0.1) # Evaluate every 10% of the data
print("Training sentences: {}".format(len(train_samples)))
print('evaluation_steps:',evaluation_steps)
print("Warmup-steps: {}".format(warmup_steps))
#print("Performance before training")
#print('dev_eval=',dev_evaluator(model))  # dev_eval= 0.6437529042748259
#print('test_eval=',test_evaluator(model))  # test_eval= 0.541513118418897

Training sentences: 10000
evaluation_steps: 125
Warmup-steps: 125


In [ ]:
# Train the model
now = datetime.datetime.now(tz=datetime.timezone(datetime.timedelta(hours=8))).strftime("%Y-%m-%d_%H-%M")
model_save_path = 'output/simcse-{}'.format(now)
#model_save_path = 'output/simcse-{}-{}-{}'.format(model_name, train_batch_size, now)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=dev_evaluator,
    epochs=1,
    evaluation_steps=evaluation_steps,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
    optimizer_params={'lr': 5e-5},
    use_amp=True         #Set to True, if your GPU supports FP16 cores
    )
print('dev_eval=',dev_evaluator(model))
print('test_eval=',test_evaluator(model))

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

dev_eval= 0.7749138041998945
test_eval= 0.7364700650619624


In [ ]:
# print 訓練過程
path = model_save_path + '/eval/similarity_evaluation_sts-dev_results.csv'
with open(path, "r", newline="") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        print(row)

In [ ]:
#余弦相似性
import numpy as np
embeddings1 = model.encode("熊猫躺在圆木上。")
embeddings2 = model.encode("一只熊猫躺着。")

cos_sim = embeddings1.dot(embeddings2) / (np.linalg.norm(embeddings1) * np.linalg.norm(embeddings2))
print(cos_sim)

0.84091616


In [ ]:
class MultipleNegativesRankingLoss(nn.Module):
    """
        This loss expects as input a batch consisting of sentence pairs (a_1, p_1), (a_2, p_2)..., (a_n, p_n)
        where we assume that (a_i, p_i) are a positive pair and (a_i, p_j) for i!=j a negative pair.
        For each a_i, it uses all other p_j as negative samples, i.e., for a_i, we have 1 positive example (p_i) and
        n-1 negative examples (p_j). It then minimizes the negative log-likehood for softmax normalized scores.
        This loss function works great to train embeddings for retrieval setups where you have positive pairs (e.g. (query, relevant_doc))
        as it will sample in each batch n-1 negative docs randomly.
        The performance usually increases with increasing batch sizes.
        For more information, see: https://arxiv.org/pdf/1705.00652.pdf
        (Efficient Natural Language Response Suggestion for Smart Reply, Section 4.4)
        You can also provide one or multiple hard negatives per anchor-positive pair by structering the data like this:
        (a_1, p_1, n_1), (a_2, p_2, n_2)
        Here, n_1 is a hard negative for (a_1, p_1). The loss will use for the pair (a_i, p_i) all p_j (j!=i) and all n_j as negatives.
        Example::
            from sentence_transformers import SentenceTransformer, losses, InputExample
            from torch.utils.data import DataLoader
            model = SentenceTransformer('distilbert-base-uncased')
            train_examples = [InputExample(texts=['Anchor 1', 'Positive 1']),
                InputExample(texts=['Anchor 2', 'Positive 2'])]
            train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)
            train_loss = losses.MultipleNegativesRankingLoss(model=model)
    """
    def __init__(self, model: SentenceTransformer, scale: float = 20.0, similarity_fct = util.cos_sim):
        """                            温度系数
        :param model: SentenceTransformer model
        :param scale: Output of similarity function is multiplied by scale value
        :param similarity_fct: similarity function between sentence embeddings. By default, cos_sim. Can also be set to dot product (and then set scale to 1)
        """
        super(MultipleNegativesRankingLoss, self).__init__()
        self.model = model
        self.scale = scale
        self.similarity_fct = similarity_fct
        self.cross_entropy_loss = nn.CrossEntropyLoss()


    def forward(self, sentence_features: Iterable[Dict[str, Tensor]], labels: Tensor):
        reps = [self.model(sentence_feature)['sentence_embedding'] for sentence_feature in sentence_features]
        embeddings_a = reps[0]
        embeddings_b = torch.cat(reps[1:])  #並聯

        scores = self.similarity_fct(embeddings_a, embeddings_b) * self.scale
        labels = torch.tensor(range(len(scores)), dtype=torch.long, device=scores.device)  # Example a[i] should match with b[i]
        return self.cross_entropy_loss(scores, labels)  # −x[(a_i, p_i)] + log(exp(x[(a_i, p_j])+exp(x[(a_i, p_j])+exp(x[(a_i, p_j])) for i!=j 
                    
    def get_config_dict(self):
        return {'scale': self.scale, 'similarity_fct': self.similarity_fct.__name__}


In [ ]:
#nn.CrossEntropyLoss()
import torch
import torch.nn as nn
import math

output = torch.randn(1, 5, requires_grad = True) #假设是网络的最后一层，5分类
label = torch.empty(1, dtype=torch.long).random_(5) # 0 - 4， 任意选取一个分类

print ('Network Output is: ', output)
print ('Ground Truth Label is: ', label)

score = output [0,label.item()].item() # label对应的class的logits（得分）
print ('Score for the ground truth class = ', label)

first = - score
second = 0
for i in range(5):
    second += math.exp(output[0,i])
second = math.log(second)

loss = first + second
print ('-' * 20)
print ('my loss = ', loss)

loss = nn.CrossEntropyLoss()
print ('pytorch loss = ', loss(output, label))

Network Output is:  tensor([[-1.2611, -1.1839, -1.0693, -0.8691,  0.0335]], requires_grad=True)
Ground Truth Label is:  tensor([1])
Score for the ground truth class =  tensor([1])
--------------------
my loss =  2.0535076885588244
pytorch loss =  tensor(2.0535, grad_fn=<NllLossBackward0>)
